In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd


In [ ]:
data_path = Path('X:/public/projects/BeJG_20230130_VisDetect/') #Define the path to the global parent folder within Ceph shared lab space. 
session_settings_paths = list(data_path.rglob("*session_settings.json")) # Get a list of all paths for the session settings. 
per_session_trials_paths = list(data_path.rglob("*trials.json")) # Get a list of all paths for the trials conditions for all trials in a session.
assert len(per_session_trials_paths) == len(session_settings_paths) # Check that the length of each list is the same (suggesting that each session settings file corresponds to a trials settings file).

In [57]:
# Load the data from the json files for trial and session data:
for session_idx in np.arange(len(session_settings_paths)):
# session_idx = 0
    with open(session_settings_paths[session_idx]) as session:
        session_settings = pd.Series(json.load(session)) # per session settings relevant for all trials within a session.
        
    with open(per_session_trials_paths[session_idx]) as per_session_trials: # this is per session (json file includes all the trials that occured in that session)
        per_session_trials_data = pd.Series(json.load(per_session_trials)) # output is a series where each element is a dictionary of values for a single trial.

    # extract 
    # from session settings: temporalfreq, autorewd; 
    # From trials data: trialoutcome, reactiontimes, TF, stimT, Stim1TF (baseline freq), Stim2TF (change freq).

    mouse_id = session_settings_paths[session_idx].name.split('_')[1]
    session_date = session_settings_paths[session_idx].name.split('_')[2]
    session_time = session_settings_paths[session_idx].name.split('_')[3]
    baseline_temporal_freq = session_settings.temporalfreq
    auto_rewd = session_settings.autorewd

    session_trial_outcome = []
    reaction_time = []
    session_TF_vectors = []
    baseline_TF = []
    change_TF = []

    for trial in np.arange(len(per_session_trials_data)):
        # trial = 0 #comment out after loop structuring
        trial_outcome = per_session_trials_data[trial]['trialoutcome']
        session_trial_outcome.append(trial_outcome) # a single string value of FA, Hit, Ref, Miss, Gray, abort
        if trial_outcome == 'Hit': trial_outcome = 'RT'
        reaction_time.append(per_session_trials_data[trial]['reactiontimes'][trial_outcome]) # an individual float value from a dictionary of reaction times for each one of the keys for the possible trial outcomes detailed above. 
        temporal_freq_vector = np.array(per_session_trials_data[trial]['TF']) # a list (turned into an array) of values for the temporal frequency from before grating started till the trial ended. 
        temporal_freq_filter = temporal_freq_vector > 0
        temporal_freq_vector = temporal_freq_vector[temporal_freq_filter]
        temporal_freq_vector = temporal_freq_vector[::3]
        session_TF_vectors.append(temporal_freq_vector)
        baseline_TF.append(per_session_trials_data[trial]['Stim1TF']) # the value of TF around which the baseline frequency revolves. 
        change_TF.append(per_session_trials_data[trial]['Stim2TF']) # the value of the change TF (change size out of 4,2,1.5,1.35,1.25)

    headers = ['mouse_id', 'session_date', 'session_time', 'baseline_temporal_freq','auto_rewd', 'trial_outcome', 'reaction_time', 'temporal_freq_vector', 'baseline_TF', 'change_TF']
    mouse_df = pd.DataFrame([mouse_id,session_date, session_time,baseline_temporal_freq,auto_rewd, session_trial_outcome, reaction_time, session_TF_vectors, baseline_TF, change_TF], index=headers).T

ValueError: Cannot set a DataFrame with multiple columns to the single column 0

In [44]:

mouse_df.loc[session_idx]

mouse_id                                                                001
session_date                                                       20230123
session_time                                                         113329
baseline_temporal_freq                                                    1
auto_rewd                                                                 1
trial_outcome                                        [Miss, Miss, FA, Miss]
reaction_time                                  [0.655, 0.655, 0.063, 0.655]
temporal_freq_vector      [[1.8590933774695886, 1.6159238997492913, 0.79...
baseline_TF                                                    [1, 1, 1, 1]
change_TF                                                      [2, 4, 2, 2]
Name: 0, dtype: object

In [52]:
session_trial_outcome

['Miss',
 'Miss',
 'Ref',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'abort',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Ref',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'abort',
 'abort',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Miss',
 'Hit']

In [49]:
trial

41